## Interfacing with XADC

TODO: Update code to DMA XADC

The AD2 uses I2C protocol so we have to use the I2C communication. The PS is what handles the communication over I2C devices. Our software on the ARM has to read from the device and then what I think makes sense is we have a DMA that can transfer the large amount of data between the memory and peripheral (AD2) in the PL. Then afterward the DMA will bring our data back to PS and it will go through the downsampled code.

In [ ]:
from pynq import Overlay
from pynq.lib.iic import AxiIIC


In [ ]:
ol = Overlay('receiver_arm_bitstream/design_1.bit')
ol.download()

In [ ]:
ol?

In [ ]:
iic = AxiIIC(ol.ip_dict["axi_iic_0"])

iic_data = [0x28]
rx_data = bytes(1)

addr = 0x28
iic.send(addr, iic_data, len(iic_data),1)
iic.receive(addr,rx_data,2,0)

In [ ]:
# Initialize the plot
plt.figure(figsize=(10, 6))
plt.xlabel('Time')
plt.ylabel('Received Data')
plt.title('Continuous Data Plot')

addr = 0x28 # or 0x4C for write, 0x4D for read
iic_data = [0x28]

while True:
    # Send data
    iic.send(addr, iic_data, len(iic_data), 1)
    
    # Receive data
    rx_data = bytearray(4)  # Adjusted for 32-bit data
    iic.receive(addr, rx_data, 4, 0)  # Receive 4 bytes
    
    # Get the current time
    current_time = time.time()
    
    # Convert the received data to an integer
    received_value = int.from_bytes(rx_data, byteorder='big')
    
    # Plot the latest received data point
    plt.scatter(current_time, received_value, color='b')
    plt.gcf().autofmt_xdate()
    plt.draw()
    plt.pause(0.01)
    
    # Clear the previous output
    clear_output(wait=True)
    display(plt.gcf())
    
    # Add a delay if needed
    time.sleep(0.5)  # Adjust the delay time as per your requirement

## Interfacing with DMA to send information to Receiver IP Block

In [1]:
from pynq import Overlay
import numpy as np
from pynq import allocate
from pynq.lib import dma
from scipy.linalg import dft
import matplotlib.pyplot as plt

In [2]:
ol=Overlay('receiver_arm_bitstream/design_1.bit')

In [3]:
ol?

In [8]:
dma1=ol.axi_dma_0
dma2=ol.axi_dma_1

In [16]:
NUM_SAMPLES=236
in_I = allocate(shape=(1,), dtype=np.float32) 
out_I = allocate(shape=(NUM_SAMPLES,), dtype=np.float32) 
out_Q = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)

in_I[0] = 5.0
    

In [18]:
receiver_ip = ol.receiver_0
receiver_ip.write(0x00,1)
# dma3.recvchannel.start()  
status_rx1 = dma1.recvchannel.running
status_rx2 = dma2.recvchannel.running
status_tx1 = dma1.sendchannel.running
# dma2.recvchannel.start()
# dma1.recvchannel.start()
# dma1.sendchannel.start()
# dma3.sendchannel.start()

# dma3.recvchannel.transfer(out_Q) 
for i in range(0,NUM_SAMPLES):
    dma1.sendchannel.transfer(in_I)
    dma2.recvchannel.transfer(out_Q)
    dma1.recvchannel.transfer(out_I)

# dma3.recvchannel.wait()
    dma2.recvchannel.wait()
    dma1.recvchannel.wait()
    dma1.sendchannel.wait()
    print(out_I)
    print(out_Q)


RuntimeError: DMA channel not idle

In [9]:
print(out_I)
print(out_I.shape)
print(out_Q)
print(out_Q.shape)

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
(236,)
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 

In [ ]:
plt.plot(out_I, "r")
plt.plot(out_Q, "b")
plt.show()

## Get output of DMA to C++ downsample ARM code

In [ ]:
#call C function